In [100]:
import requests
from concurrent.futures import ThreadPoolExecutor
import csv
import pandas as pd
import re
from bs4 import BeautifulSoup

In [ ]:
query_params = {
    'text': 'программист',
    'area': 2, 
    'per_page': 100,
}

initial_response = requests.get('https://api.hh.ru/vacancies', params=query_params)
vacancies_json = initial_response.json()['items']

thread_num = 10

filename = 'vacs.csv'

def process_vacancy(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    response = requests.get(url)
    if response.status_code == 200:
        vacancy_data = response.json()
        record = {
                'title': vacancy_data['name'],
                'description': vacancy_data['description'],
                'city': vacancy_data['area']['name'],
                'metro_station': (
                    vacancy_data['address'].get('metro', {}).get('station_name') 
                    if vacancy_data.get('address') and vacancy_data['address'].get('metro') 
                    else None
                ),
                'min_salary': vacancy_data['salary']['from'] if vacancy_data['salary'] else None,
                'max_salary': vacancy_data['salary']['to'] if vacancy_data['salary'] else None,
                'employment_type': vacancy_data['employment']['name'],
                'employer': vacancy_data['employer']['name']
            }
        return record
    else:
        print(f"Ошибка при получении данных о вакансии {vacancy_id}: {response.status_code}")
        return None

In [ ]:
vacancy_ids = [item['id'] for item in vacancies_json]

with open(filename, mode='w', newline='', encoding='utf-8') as file:
    field_names = ['title', 'description', 'city', 'metro_station', 'min_salary', 'max_salary', 'employment_type', 'employer']
    writer = csv.DictWriter(file, fieldnames=field_names)
    writer.writeheader()

    with ThreadPoolExecutor(max_workers=thread_num) as executor:
        futures = [executor.submit(process_vacancy, vid) for vid in vacancy_ids]

        for future in futures:
            result = future.result()
            if result:
                writer.writerow(result)

In [96]:
data = pd.read_csv('vacs.csv')
data

,title,description,city,metro_station,min_salary,max_salary,employment_type,employer
0,Python developer (Middle),<h2><strong>Мечтай о большем с Napoleon IT</st...,Санкт-Петербург,NaN,NaN,NaN,Полная занятость,Napoleon IT
1,Java разработчик,"<p>* обращаем внимание, что услуги для соискат...",Санкт-Петербург,NaN,NaN,NaN,Полная занятость,IT-Work
2,Программист (Junior - младший разработчик),<p>В нашу команду требуется программист.</p> <...,Санкт-Петербург,Старая Деревня,50000.0,70000.0,Полная занятость,ПТМК
3,PHP-разработчик,"<p>Команда, где уделяют должное внимание перед...",Санкт-Петербург,NaN,NaN,NaN,Полная занятость,Настоящая Статистика
4,Стажер Java разработчик,<p>Мы развиваем продукт – платформу удаленного...,Санкт-Петербург,NaN,NaN,NaN,Частичная занятость,КРОК
...,...,...,...,...,...,...,...,...
95,Программист 1С,<strong>Функционал должности:</strong> <ul> <l...,Санкт-Петербург,Московские ворота,NaN,NaN,Полная занятость,Типография Индустрия Цвета
96,Инженер-программист ПЛК (PLC),<strong>Обязанности:</strong> <ul> <li> <p>Раз...,Санкт-Петербург,Бухарестская,170000.0,NaN,Полная занятость,Альфа Л Сервис
97,Java-разработчик (Senior),<p><strong>Мы</strong> — IT-компания <strong>N...,Санкт-Петербург,NaN,NaN,NaN,Полная занятость,NAUKA
98,Младший разработчик/программист 1С,<div><strong>SIGIR group — </strong>федеральны...,Санкт-Петербург,Выборгская,NaN,NaN,Полная занятость,Sigir Group


In [101]:
def preprocess(txt):
    txt = re.sub(r'<.*?>', '', txt)
    txt = BeautifulSoup(txt, "html.parser").get_text()
    return txt

In [103]:
data['description'] = data['description'].apply(preprocess)
data

,title,description,city,metro_station,min_salary,max_salary,employment_type,employer
0,Python developer (Middle),Мечтай о большем с Napoleon IT Мы — не просто ...,Санкт-Петербург,NaN,NaN,NaN,Полная занятость,Napoleon IT
1,Java разработчик,"* обращаем внимание, что услуги для соискателе...",Санкт-Петербург,NaN,NaN,NaN,Полная занятость,IT-Work
2,Программист (Junior - младший разработчик),В нашу команду требуется программист. Требован...,Санкт-Петербург,Старая Деревня,50000.0,70000.0,Полная занятость,ПТМК
3,PHP-разработчик,"Команда, где уделяют должное внимание передаче...",Санкт-Петербург,NaN,NaN,NaN,Полная занятость,Настоящая Статистика
4,Стажер Java разработчик,Мы развиваем продукт – платформу удаленного до...,Санкт-Петербург,NaN,NaN,NaN,Частичная занятость,КРОК
...,...,...,...,...,...,...,...,...
95,Программист 1С,Функционал должности: Сопровождение конфигура...,Санкт-Петербург,Московские ворота,NaN,NaN,Полная занятость,Типография Индустрия Цвета
96,Инженер-программист ПЛК (PLC),Обязанности: Разработка программного обеспеч...,Санкт-Петербург,Бухарестская,170000.0,NaN,Полная занятость,Альфа Л Сервис
97,Java-разработчик (Senior),Мы — IT-компания NAUKA. Занимаемся разработкой...,Санкт-Петербург,NaN,NaN,NaN,Полная занятость,NAUKA
98,Младший разработчик/программист 1С,SIGIR group — федеральный мультимодальный логи...,Санкт-Петербург,Выборгская,NaN,NaN,Полная занятость,Sigir Group


In [111]:
data.isna().sum()

title               0
description         0
city                0
metro_station      45
min_salary         50
max_salary         64
employment_type     0
employer            0
dtype: int64

In [114]:
data['employment_type'] = data['employment_type'].replace(
    {
        'Полная занятость': 0,
        'Частичная занятость': 1
    }
)

data

,title,description,city,metro_station,min_salary,max_salary,employment_type,employer
0,Python developer (Middle),Мечтай о большем с Napoleon IT Мы — не просто ...,Санкт-Петербург,NaN,NaN,NaN,0,Napoleon IT
1,Java разработчик,"* обращаем внимание, что услуги для соискателе...",Санкт-Петербург,NaN,NaN,NaN,0,IT-Work
2,Программист (Junior - младший разработчик),В нашу команду требуется программист. Требован...,Санкт-Петербург,Старая Деревня,50000.0,70000.0,0,ПТМК
3,PHP-разработчик,"Команда, где уделяют должное внимание передаче...",Санкт-Петербург,NaN,NaN,NaN,0,Настоящая Статистика
4,Стажер Java разработчик,Мы развиваем продукт – платформу удаленного до...,Санкт-Петербург,NaN,NaN,NaN,1,КРОК
...,...,...,...,...,...,...,...,...
95,Программист 1С,Функционал должности: Сопровождение конфигура...,Санкт-Петербург,Московские ворота,NaN,NaN,0,Типография Индустрия Цвета
96,Инженер-программист ПЛК (PLC),Обязанности: Разработка программного обеспеч...,Санкт-Петербург,Бухарестская,170000.0,NaN,0,Альфа Л Сервис
97,Java-разработчик (Senior),Мы — IT-компания NAUKA. Занимаемся разработкой...,Санкт-Петербург,NaN,NaN,NaN,0,NAUKA
98,Младший разработчик/программист 1С,SIGIR group — федеральный мультимодальный логи...,Санкт-Петербург,Выборгская,NaN,NaN,0,Sigir Group


In [115]:
data.to_csv('prepared_vacs.csv')